In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import json
from haversine import haversine

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

In [2]:
# Use the SOTF from Bruce on 0927
df_SOTF_number=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Maggie_cities_for_SOTF_20190927/BL SOTF as of Sep 27 2019 BR.xlsx")
# set_SOTF_number.sheet_names
df_SOTF_number=df_SOTF_number.parse("SOTF pre 9-27-19 (451)",dtype=str)
df_SOTF_number['Open Date']=pd.to_datetime(df_SOTF_number['Open Date']).dt.date


df_SOTF_number=df_SOTF_number[(df_SOTF_number['SOTF']=="Yes") & (pd.notnull(df_SOTF_number['Open Date']))]
df_SOTF_number=df_SOTF_number[df_SOTF_number['Open Date']<=datetime.date(2019,10,1)]
df_SOTF_number=df_SOTF_number[df_SOTF_number['Store'].apply(lambda x: x.isdigit())]
SOTF_list=df_SOTF_number['Store'].unique().tolist()
len(SOTF_list)

451

In [3]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190901-135942-508.txt",
                        sep="|",dtype=str)
store_list=store_list[store_list['location_id'].isin(SOTF_list)]

In [4]:
[x for x in SOTF_list if x not in store_list['location_id'].tolist()]

['4678', '4702', '4703', '4705', '4706', '5402', '5407']

In [5]:
Google_Address=dict()

#'4678' 4678 	3501 EAST LINCOLNWAY	CHEYENNE PLAZA	CHEYENNE	WY	82001-6302
Google_Address.update({"4678":{"address":"3501 E Lincolnway, Cheyenne, WY 82001","coor":[41.1370665,-104.7673948]}})

#'4702' 4702 	6375 MONTANA AVENUE	KMART SHOPPING CENTER	EL PASO	TX	79925-2044
Google_Address.update({"4702":{"address":"6375 Montana Ave, El Paso, TX 79925","coor":[31.7898615,-106.4049886]}})

#'4703' 4703 	7370 NORTH BLACKSTONE AVENUE		FRESNO	CA	93650-1231
Google_Address.update({"4703":{"address":"7370 N Blackstone Ave, Fresno, CA 93650","coor":[36.8439621,-119.7892021]}})

#'4705' 4705 	370 AUTO CENTER CIRCLE	NORTHRIDGE MALL	SALINAS	CA	93907
Google_Address.update({"4705":{"address":"370 Auto Center Cir, Salinas, CA 93906","coor":[36.7148566,-121.6607111]}})

#'4706' 4706 	3170 CHILI AVE UNIT # T5	CHILI PLAZA	ROCHESTER	NY	14624-5410
Google_Address.update({"4706":{"address":"3170 Chili Ave T5, Rochester, NY 14624","coor":[43.1085526,-77.7466991]}})

#'5402' 5402 	14603 TELEGRAPH ROAD	TOYS R US CENTER	WOODBRIDGE	VA	22192-4601
Google_Address.update({"5402":{"address":"14603 Telegraph Rd, Woodbridge, VA 22194","coor":[38.6373886,-77.2914399]}})

#'5407' 5407 	14154 EAST WADE HAMPTON BLVD	COLLINS CENTER	GREER	SC	29651-1554
Google_Address.update({"5407":{"address":"14154 E Wade Hampton Blvd, Greer, SC 29651","coor":[34.9489239,-82.211875]}})



In [6]:
stores_zip_labels=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
print(stores_zip_labels.sheet_names)

stores_zip_labels=stores_zip_labels.parse("view_by_store",dtype=str)
stores_zip_labels=stores_zip_labels[stores_zip_labels['location_id'].isin(SOTF_list)]
stores_zip_labels['all_trans_P_zips']=stores_zip_labels['all_trans_P_zips'].apply(eval)
stores_zip_labels['all_trans_S_zips']=stores_zip_labels['all_trans_S_zips'].apply(eval)
stores_zip_labels['zips_in_10']=stores_zip_labels['zips_in_10'].apply(eval)


['unique_zips_full_footprint', 'view_by_TA', 'view_by_store', 'view_for_Tableau']


In [7]:
old_sotf_df=pd.DataFrame()

for ind,row in stores_zip_labels.iterrows():
    store_id=row['location_id']
    df_P=pd.DataFrame({"zip_cd":row['all_trans_P_zips']},index=["P"]*len(row['all_trans_P_zips']))
    df_S=pd.DataFrame({"zip_cd":row['all_trans_S_zips']},index=["S"]*len(row['all_trans_S_zips']))
    df_10=pd.DataFrame({"zip_cd":row['zips_in_10']},index=["zips_in_10"]*len(row['zips_in_10']))
    df_store=pd.concat([df_P,df_S,df_10])
    df_store['location_id']=store_id
    df_store=df_store.reset_index().rename(columns={"index":"trans_flag"})
    old_sotf_df=old_sotf_df.append(df_store)

In [9]:
old_sotf_df['zip_cd'].nunique()

4726

In [10]:
old_sotf_df[old_sotf_df['trans_flag']!="S"].shape

(2044, 3)

In [11]:
new_sotf=[x for x in SOTF_list if x not in stores_zip_labels['location_id'].tolist()]

In [12]:
new_sotf_df=pd.DataFrame({"location_id":new_sotf})
new_sotf_df=pd.merge(new_sotf_df,store_list,on="location_id",how="left")
new_sotf_df=new_sotf_df[['location_id','latitude_meas','longitude_meas']]
new_sotf_df['latitude_meas']=new_sotf_df['latitude_meas'].astype(float)
new_sotf_df['longitude_meas']=new_sotf_df['longitude_meas'].astype(float)
new_sotf_df['latitude_meas']=new_sotf_df['location_id'].apply(lambda x: Google_Address[x]['coor'][0] if x in Google_Address.keys() else new_sotf_df.loc[new_sotf_df["location_id"]==x,"latitude_meas"].values[0])
new_sotf_df['longitude_meas']=new_sotf_df['location_id'].apply(lambda x: Google_Address[x]['coor'][1] if x in Google_Address.keys() else new_sotf_df.loc[new_sotf_df["location_id"]==x,"longitude_meas"].values[0])
new_sotf_df=new_sotf_df.reset_index()
del new_sotf_df['index']

In [13]:
new_store_zips=pd.DataFrame()
for ind, row in new_sotf_df.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])

    for zip_cd,zip_coor in zip_centers.items():
        dist=haversine(store_center,zip_coor,miles=True)
        if dist<=10:
            df=pd.DataFrame({"zip_cd":zip_cd},index=[row['location_id']])
            df=df.reset_index().rename(columns={"index":"location_id"})
            new_store_zips=new_store_zips.append(df)
new_store_zips['trans_flag']="zips_in_10"

In [16]:
df_output=new_store_zips.append(old_sotf_df)
df_output_no_S=df_output[df_output['trans_flag']!="S"]

In [17]:
df_unique_zip_all=df_output[['zip_cd']].drop_duplicates()
df_unique_zip_P_10=df_output_no_S[['zip_cd']].drop_duplicates()


In [18]:
writer=pd.ExcelWriter("./BL_zips_for_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zip_P_10.to_excel(writer,"unique_zip_P_or_10new",index=False)
df_unique_zip_all.to_excel(writer,"unique_zip_all_SOTF",index=False)
df_output_no_S.to_excel(writer,"zip_SOTF_no_S",index=False)
df_output.to_excel(writer,"zip_SOTF_all",index=False)
writer.save()